# G-Research Crypto Forecasting | "data leakage" | Public Leaderboard = 0.9999

### Acknowledgements
My acknowledgments are given to: [dataista0 (Julián Peller). 🪙💲 G-Research- Using the overlap fully [LB=0.99]](https://www.kaggle.com/julian3833/g-research-using-the-overlap-fully-lb-0-99). I saw his public notebook and decided to copy and try by myself.

> As author says, "the test period is contained in the train data to get the closest point for each test row. It is not actually leaky because the fact that the LB is part of the public data was publicly disclosed by the organizers."

As you can see, this code doesn't include any model training. And, I think, that it will not suitable for private leaderboard, if it will be calculated with another set of data. That is why it is just an example, how to submit to the competition without any difficult work. 

In [ ]:
import numpy as np
import pandas as pd
import gresearch_crypto

In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

In [ ]:
df = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv', 
                 usecols=['Target', 
                          'Asset_ID',
                          'timestamp'], 
                 dtype={'Asset_ID': 'int8'})

df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')

df = df.set_index('datetime').drop('timestamp', axis=1)
df = df[(df.index.year == 2021) & (df.index.month > 5)]

dfs = {asset_id: df[df['Asset_ID'] == asset_id].resample('1min').interpolate().copy() for asset_id in df['Asset_ID'].unique()}

del df

In [ ]:
for df_test, df_pred in iter_test:
    df_test['datetime'] = pd.to_datetime(df_test['timestamp'], unit='s')
    for _, row in df_test.iterrows():
        try:
            df = dfs[row['Asset_ID']]
            closest_train_sample = df.iloc[df.index.get_loc(row['datetime'], method='nearest')]
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = closest_train_sample['Target']
        except:
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
    df_pred['Target'] = df_pred['Target'].fillna(0)
    env.predict(df_pred)